In [ ]:
%matplotlib inline

In [ ]:
import cdsapi
import os
import urllib
import pathlib
import argparse
from datetime import datetime
import multiprocessing
from multiprocessing.pool import Pool

In [ ]:
import numpy as np
import pandas as pd
import xarray as xr

In [ ]:
# grib_params = {'engine':'cfgrib', 'backend_kwargs':{'indexpath':''}}

In [ ]:
dates = pd.date_range(start='1997-03-01', end='2019-12-01', freq='MS')

In [ ]:
opath = pathlib.Path('/media/nicolasf/END19101/data/GCMs/downloads/ECMWF/PRECIP/')

In [ ]:
if not opath.exists():
    opath.mkdir(parents=True)

In [ ]:
for date in dates: 
    
    year = date.year 
    
    month = date.month 
    
    print(f"downloading and processing {year} {str(month).zfill(2)}")
    
    while not opath.joinpath(f'CDS_ECMWF_PRECIP_{year}_{str(month).zfill(2)}.nc').exists():
    
        
        c = cdsapi.Client(timeout=60, sleep_max=30)
        
        
        for step in range(1, 7):

            data = c.retrieve(
                'seasonal-monthly-single-levels',
                {
                    'originating_centre': 'ecmwf',
                    'system': '5',
                    'variable': 'total_precipitation',
                    'product_type': 'monthly_mean',
                    'year': str(year),
                    'month': str(month).zfill(2),
                    'leadtime_month': str(step),
                    'format': 'netcdf',
                },
                f'{str(opath)}/download_{step}.nc')

            data.delete()

        lfiles = list(opath.glob("download_?.nc"))

        lfiles.sort()

        dset = xr.open_mfdataset(lfiles, concat_dim=['time']) 

        dset = dset.rename({'time':'step'})

        dset['time'] = (('time'), [datetime(year,month,1)])

        dset.to_netcdf(opath.joinpath(f'CDS_ECMWF_PRECIP_{year}_{str(month).zfill(2)}.nc'))

        if opath.joinpath(f'CDS_ECMWF_PRECIP_{year}_{str(month).zfill(2)}.nc').exists(): 
            print(f"{str(opath.joinpath(f'CDS_ECMWF_PRECIP_{year}_{str(month).zfill(2)}.nc'))} has been saved OK ...")

            [fname.unlink(missing_ok=False) for fname in lfiles] 

        dset.close()
        